# CircuitsVis: extensions

This notebook contains my fork of circuitsvis. It includes a few extra features relative to the standard version, such as:

* It can be run on a TransformerLens `ActivationCache` rather than just on a tensor of attention patterns (all examples in the notebook do this),
* Option to show value-weighted attention patterns rather than regular patterns,
* It can toggle between multiple sequences in a batch,
* Plots can be opened in browser by default rather than displayed inline (won't work in Colab),
* Added [bertviz](https://github.com/jessevig/bertviz)-style plots, with some extra features.

Please comment or send me a message if you have any feedback. I hope you find this useful!

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/CallumMcDougall_circuit_visualisations_colorful_clear_simple_bo_6c1b2148-84e2-4ab8-906e-d104cdc85e06.png" width="320">

# Setup code (don't read, just run)

In [1]:
try:
    import google.colab
    # %pip install jaxtyping
    %pip install transformer_lens
    %pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
except:
    import os; os.environ["ACCELERATE_DISABLE_RICH"] = "1"
    from IPython import get_ipython
    ipython = get_ipython()
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

import torch as t
from torch import Tensor
import circuitsvis as cv
from jaxtyping import Float
from IPython.display import clear_output, display
from transformer_lens import HookedTransformer

t.set_grad_enabled(False)

gpt2 = HookedTransformer.from_pretrained("gpt2-small")
clear_output()

In [2]:
sentence0 = "When Mary and John went to the shops, John gave a drink to Mary."
sentence1 = "When Mary and John went to the shops, Mary gave a drink to John."
sentence2 = "The cat sat on the mat."
sentences_all = [sentence0, sentence1, sentence2]

names_filter = lambda name: any(name.endswith(f"hook_{s}") for s in ["pattern", "q", "k", "v"])
logits, cache = gpt2.run_with_cache(sentence0, names_filter=names_filter, remove_batch_dim=True)
logits_all, cache_all = gpt2.run_with_cache(sentences_all, names_filter=names_filter)
logits, cache_full = gpt2.run_with_cache(sentence0, remove_batch_dim=True)

attn: Float[Tensor, "layers heads seq_Q seq_K"] = t.stack([cache["pattern", i] for i in range(gpt2.cfg.n_layers)])
attn_all: Float[Tensor, "batch layers heads seq_Q seq_K"] = t.stack([cache_all["pattern", i] for i in range(gpt2.cfg.n_layers)], dim=1)

tokens = gpt2.to_str_tokens(sentence0)
tokens_all = gpt2.to_str_tokens(sentences_all)

# Overview of function



The main function is `cv.attention.from_cache`. The most important arguments are:

* `cache` - the `ActivationCache` object. This has to contain the appropriate activations (i.e. `pattern`, plus `v` if you're using value-weighted attention, plus `q` and `k` if you're using `lines` mode).
* `tokens` - either a list of strings (if batch size is 1), or a list of lists of strings (if batch size is > 1).



The optional arguments are:

* `heads` - if specified (e.g. `[(9, 6), (9, 9)]`), these heads will be shown in the visualisation. If not specified, behaviour is determined by the `layers` argument.
* `layers` - this can be an int (= single layer), list of ints (= list of layers), or None (= all layers). If `heads` are not specified, then the value of this argument determines what heads are shown.
* `batch_idx` - if the cache has a batch dimension, then you can specify this argument (as either an int, or list of ints). Note that you can have nontrivial batch size in your visualisations (you'll be able to select different sequences using a dropdown).
* `use_value_weighted_attn` - if True, then the visualisation will use value-weighted attention, i.e. every attention probability $A^h[s_Q, s_K]$ will be replaced with:

$$
A^h[s_Q, s_K] \times \frac{\|v^h[s_K]\|}{\underset{s}{\max} \|v^h[s]\|}
$$

* `mode` - this can be "large", "small" or "lines", for producing the three different types of attention plots (see below for examples of all).
* `return_mode` - this can be "browser" (open plot in browser; doesn't work in Colab or VMs), "html" (returns html object), or "view" (displays object inline).
* `radioitems` - if True, you select the sequence in the batch using radioitems rather than a dropdown. Defaults to False.
* `title` - if given, then a title is appended to the start of your plot (i.e. an `<h1>` HTML item).
* `help` - if True, prints out a string explaining the visualisation. Defulats to False.

# Examples

Below is a set of examples, along with some brief explanations.

## Default arguments (batch vs no batch)



The first 2 examples below show how the function works when you don't specify any other keyword arguments (for cache having a batch dim vs not having a batch dim).

The third shows how you can also specify a single element in a batch, using the `batch_idx` argument.

In [ ]:
cv.attention.from_cache(
    cache = cache,
    tokens = tokens,
)

In [ ]:
cv.attention.from_cache(
    cache = cache_all,
    tokens = tokens_all,
)

In [ ]:
cv.attention.from_cache(
    cache = cache_all,
    tokens = tokens_all,
    batch_idx = 1, # Different way to speciful a sequence within batch
)

## Different ways to specify heads



We can specify heads using the `layer` argument, or the `heads` argument.

In [ ]:
cv.attention.from_cache(
    cache = cache,
    tokens = tokens,
    layers = [9, 10],
)

In [ ]:
cv.attention.from_cache(
    cache = cache,
    tokens = tokens,
    heads = [(9, 6), (9, 9), (10, 0)], # Showing all the name mover heads: `to` attends to the IO token `Mary`
)

## Different modes: `"large"` and `"lines"`



The mode above is "small" (also known as `attention_patterns` in circuitsvis). You can also use "large" (which is `attention_heads` in circuitsvis), or "lines" (which is like the neuron view in [bertviz](https://github.com/jessevig/bertviz), but with a few extra features I added).

In [ ]:
cv.attention.from_cache(
    cache = cache_all,
    tokens = tokens_all,
    heads = [(9, 6), (9, 9), (10, 0)],
    mode = "large",
)

In [ ]:
cv.attention.from_cache(
    cache = cache,
    tokens = tokens,
    mode = "lines",
    display_mode = "dark", # Can also choose "light"
)

In [ ]:
cv.attention.from_cache(
    cache = cache,
    tokens = tokens,
    heads = [(9, 6), (9, 9), (10, 0)],
    mode = "lines",
)

## Value-weighted attention


Value-weighted attention is a pretty neat concept. TL;DR - **if a value vector doubled in magnitude but attention probability halved then nothing would change; so we should expect the attention probability to be more meaningful once we scale by the magnitude of the value vector**. This is what the `use_value_weighted_attn` argument does; every attention probability $A^h[s_Q, s_K]$ will be replaced with:

$$
A^h[s_Q, s_K] \times \frac{\|v^h[s_K]\|}{\underset{s}{\max} \|v^h[s]\|}
$$

where $\|v^h[s]\|$ is the $L_2$ norm of the value vector at source position $s$ in head $h$.

In particular, we see that the attention on the BOS token is much lower (because usually this token is attended to as a placeholder, and not much is actually copied). The example below shows GPT2-small's 3 main **name mover heads**; they attend from the `to` token (as well as some other places) to the first `Mary` token, and copy it as a prediction. We can see from these patterns that this is basically all that heads `9.6` and `9.9` do (`10.7` is a little messier but still roughly the same result holds).

In [ ]:
cv.attention.from_cache(
    cache = cache,
    tokens = tokens,
    heads = [(9, 6), (9, 9), (10, 0)], # showing all the name mover heads: `to` attends to the IO token `Mary`
    attention_type = "info-weighted", # or try "value-weighted"
)

## Other arguments

`title` (optional) specifies a title.

If `help` is True, then a string explaining the visualisation is printed out (as well as an explanation of the non-default arguments which you're using).

If `radioitems` is True, then you select different sequences in the batch using radioitems rather than a dropdown. Defaults to False.

`display_mode` can be "dark" (default) or "light". This only affects the "lines" mode.

`return_mode` can be "browser" (open in browser), "html" (return html object), or "view" (display object inline; this is default).

Note that the browser view is often preferable - it doesn't slow down your IDE, and it can reduce flickering when you switch between different sequences in your batch. However, this won't always work (e.g. in virtual machines or on Colab). In this case, you should use `return_mode = "html"`, then save the result and download & open it manually. Example code is given below.

In [ ]:
html_object = cv.attention.from_cache(
    cache = cache_all,
    tokens = tokens_all,
    batch_idx = 0,
    heads = [(9, 6), (9, 9), (10, 0)],
    mode = "lines",
    title = "Attention of name mover heads (lines mode)",
    return_mode = "html",
    help = True,
    radioitems = True,
    display_mode = "light", # This might be better if you're opening in browser
)

display(html_object)

with open("my_file.html", "w") as f:
    f.write(html_object.data)

# Attribution plots (work-in-progress!)

Attention plots might also be decent for logit attribution. The values of each cell are "logits directly written in the correct direction".

You can pass the `resid_directions` argument to the function, and it'll measure the attribution in that direction (e.g. this could be a `logit_diff` vector). Note that it also supports `resid_directions` being a vector of length `d_vocab`; then it can convert this into a vector in the residual stream by mapping it backwards through `W_U` (and this also means we can do logit attribution for the unembedding bias `b_U`).

There are 2 really hacky things about this function, which are why I don't recommend people use it yet.

1. The colors are awkward; lots of translations and scalings have to be done to make sure that (1) the whitepoint is zero logit attribution and (2) no values are more than 1. The way I did this is by dividing all the component logit attributions by the maximum absolute value of all components' contributions over all sequence positions (which leads to a sparser & cleaner plot, and also makes it easier to make relative comparisons between different values). For the large plot `attention_heads`, this was sufficient, because it also supports negative values: `[-1, 0]` is red and `[0, 1]` is blue. For the smaller plot `attention_patterns`, there's only one color shade for each facet plot, and so this one has to be split up into 2 separate plots.

2. If you're just doing attribution at a particular sequence position, it makes sense to just have a `(1, seq_len)`-shape plot per component, rather than a `(seq_len, seq_len)`-size plot where you only care about one row. Currently, circuitsvis doesn't support having different source and destination tokens so this isn't yet possible. A hacky solution: whenever `resid_directions` is just a single vector (i.e. it doesn't have a `seq_len` dimension), I broadcast it along the square attention plots, so you see a vertical stripe rather than just a dot. Uncomment the lines below to see this in action.

I expect both of these two things to be solved eventually, but they're not high on my priority list right now.

In [ ]:
resid_directions = t.zeros(gpt2.cfg.d_vocab, dtype=t.float32, device=gpt2.cfg.device)
resid_directions[gpt2.to_single_token(" John")] = -1
resid_directions[gpt2.to_single_token(" Mary")] = +1

end_token_position = len(tokens) - 1 - tokens[::-1].index(" to")

html_pos, html_neg = cv.attribution.from_cache(
    model = gpt2,
    cache = cache_full,
    tokens = tokens,
    mode = "small",
    # seq_pos = end_token_position,
    # resid_directions = resid_directions,
)
display(html_pos)
display(html_neg)

In [ ]:
html_all = cv.attribution.from_cache(
    model = gpt2,
    cache = cache_full,
    tokens = tokens,
    mode = "large",
    # seq_pos = end_token_position,
    # resid_directions = resid_directions,
)
display(html_all)